<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/yvp_species_richness_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentation

[Readme fixed plot vegetation data](https://docs.google.com/document/d/16-Aq8u9Rudd78fSzfjvpCXyQgE-BstC-d2PjYfmLtcw/edit?usp=sharing)

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.2     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [2]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’




# Source

In this view of the yvp data, species from the cover-based and additional species summaries will be vertically combined for each grid point. Since the additional species summary records species presence only, this view will be limited entirely to species presence. The result will be the plant species richness for each grid point, and this is useful for comparing plant communities, finding the locations of rarer species, or identifying grid points where non-native species are just getting established. After these data are processed, we will want to retain knowledge of whether a species was detected during point-intercept of additional species surveys so that we can evaluate the potential rarity of a given species. The new variable detection_type will allow us to do this.

## Database Connection

In [3]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [4]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [5]:
billing <- bq_test_project()

### yvp_vegetation_cover

In [6]:
sql_vegetation_cover <- 
"
SELECT
  CONCAT(plot_code, \" \", date) AS survey_code,
  plot_code,
  SUBSTR(SAFE_CAST(date AS STRING), 0, 4) AS year,
  plot_loc,
  plot_rep,
  plot_num,
  (\"cover_est\") AS detection_type,
  species_key
FROM
  `mpg-data-warehouse.vegetation_fixed_plot_yvp.yvp_vegetation_cover`
WHERE
  cover_pct != 0
"

In [7]:
bq_vegetation_cover <- bq_project_query(billing, sql_vegetation_cover)

In [8]:
tb_vegetation_cover <- bq_table_download(bq_vegetation_cover)

In [9]:
df_vegetation_cover <- as.data.frame(tb_vegetation_cover)

In [10]:
df_vegetation_cover %>% glimpse() 

Rows: 19,994
Columns: 8
$ survey_code    <chr> "YVP N7 2017-06-08", "YVP N7 2017-06-08", "YVP N7 2017…
$ plot_code      <chr> "YVP N7", "YVP N7", "YVP N7", "YVP N7", "YVP N7", "YVP…
$ year           <chr> "2017", "2017", "2017", "2017", "2017", "2017", "2017"…
$ plot_loc       <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N",…
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, …
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "c…
$ species_key    <int> 82, 113, 153, 187, 233, 266, 286, 320, 389, 411, 437, …


### yvp_additional_species

In [11]:
sql_additional_species <- "
SELECT 
  CONCAT(plot_code, \" \", date) AS survey_code,
  plot_code,
  SUBSTR(SAFE_CAST(date AS STRING), 0, 4) AS year,
  plot_loc,
  plot_rep,
  plot_num,
  (\"supplemental_obs\") AS detection_type,
  species_key
FROM
  `mpg-data-warehouse.vegetation_fixed_plot_yvp.yvp_additional_species`
"

In [12]:
bq_additional_species <- bq_project_query(billing, sql_additional_species)

In [13]:
tb_additional_species <- bq_table_download(bq_additional_species)

In [14]:
df_additional_species <- as.data.frame(tb_additional_species)

In [15]:
df_additional_species %>% glimpse()

Rows: 1,280
Columns: 8
$ survey_code    <chr> "YVP 12 2018-07-10", "YVP 12 2018-07-10", "YVP 12 2018…
$ plot_code      <chr> "YVP 12", "YVP 12", "YVP 12", "YVP 12", "YVP 12", "YVP…
$ year           <chr> "2018", "2018", "2018", "2018", "2018", "2018", "2018"…
$ plot_loc       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 12, 12, 12, 12, 12, 246, 246, 246, 246, 571, 571, 571,…
$ detection_type <chr> "supplemental_obs", "supplemental_obs", "supplemental_…
$ species_key    <int> 72, 179, 426, 63, 484, 402, 53, 274, 334, 496, 240, 56…


### location_position_classification

In [16]:
sql_position_classification <- "
SELECT
  grid_point,
  aspect_mean_deg,
  elevation_mean_m,
  slope_mean_deg,
  cover_type_2016_gridVeg,
  type3_vegetation_indicators,
  type4_indicators_history
FROM
  `mpg-data-warehouse.grid_point_summaries.location_position_classification`
"

In [17]:
bq_position_classification <- bq_project_query(billing, sql_position_classification)

In [18]:
tb_position_classification <- bq_table_download(bq_position_classification)

In [19]:
df_position_classification <- as.data.frame(tb_position_classification)

In [20]:
df_position_classification %>% glimpse()

Rows: 582
Columns: 7
$ grid_point                  <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13…
$ aspect_mean_deg             <dbl> 334.7050, 45.3030, 221.3340, 290.4890, 28…
$ elevation_mean_m            <dbl> 1395.64, 1456.09, 1126.90, 1166.33, 1179.…
$ slope_mean_deg              <dbl> 28.44230, 12.22630, 4.25130, 2.68361, 4.2…
$ cover_type_2016_gridVeg     <chr> "woodland/forest", "non-irrigated grassla…
$ type3_vegetation_indicators <chr> "mixed canopy conifer", "uncultivated gra…
$ type4_indicators_history    <chr> "mixed canopy conifer", "uncultivated gra…


### vegetation_species_metadata

In [21]:
sql_species_metadata <- "
SELECT
  key_plant_species,
  key_plant_code,
  plant_name_sci,
  plant_name_common,
  plant_native_status,
  plant_life_cycle,
  plant_life_form
FROM
  `mpg-data-warehouse.vegetation_species_metadata.vegetation_species_metadata`"

In [22]:
bq_species_metadata <- bq_project_query(billing, sql_species_metadata)

In [23]:
tb_species_metadata <- bq_table_download(bq_species_metadata)

In [24]:
df_species_metadata <- as.data.frame(tb_species_metadata)

In [25]:
df_species_metadata %>% glimpse()

Rows: 754
Columns: 7
$ key_plant_species   <int> 360, 13, 26, 53, 738, 75, 76, 746, 83, 88, 86, 87…
$ key_plant_code      <chr> "NV", "AGRSCA", "ANDGER", "ARIPUR", "BOUCUR", "BO…
$ plant_name_sci      <chr> "no vegetation", "Agrostis scabra", "Andropogon g…
$ plant_name_common   <chr> "no vegetation", "rough bentgrass", "big bluestem…
$ plant_native_status <chr> "none", "native", "native", "native", "native", "…
$ plant_life_cycle    <chr> "unknown", "perennial", "perennial", "perennial",…
$ plant_life_form     <chr> "none", "graminoid", "graminoid", "graminoid", "g…


# Wrangle

With the data from yvp_vegetation_cover, species lists must first be summarized as distinct key_plant_species values within survey_code values. This is because the raw data are estimated in 10 subplots per transect, and species names will often be redundant among subplots. Then the yvp_vegetation_cover data can be vertically bound to the yvp_additional_species data after some light coercion of field names.

One caution with these data. According to protocol, a plant species is only included in the additional species table if it was not found during cover-based surveys. In practice, I assume that this is routinely violated because it isn’t easy to remember all the species surveyed, nor is it efficient to check time after time. It’s important that we eliminate duplicate species for a given grid point. When duplication exists, default to detection_type = “cover_est”. This will prevent upward bias of richness estimates and will make downstream analyses less complicated. Some operation that again summarizes distinct key_plant_species values within survey_code values will be necessary. For additional information on this point, please see instructions for a similar operation with the point-intercept data in the gridVeg [Readme](https://docs.google.com/document/d/1JWnhxNjeSQZkSnGhtHP68i_l1mDj4vPFMBdUvGqN0TA/edit#heading=h.hnb7ex8jlp42).


## Remove duplicates

### yvp_vegetation_cover

In [26]:
df_vegetation_cover %>% glimpse()

Rows: 19,994
Columns: 8
$ survey_code    <chr> "YVP N7 2017-06-08", "YVP N7 2017-06-08", "YVP N7 2017…
$ plot_code      <chr> "YVP N7", "YVP N7", "YVP N7", "YVP N7", "YVP N7", "YVP…
$ year           <chr> "2017", "2017", "2017", "2017", "2017", "2017", "2017"…
$ plot_loc       <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N",…
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, …
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "c…
$ species_key    <int> 82, 113, 153, 187, 233, 266, 286, 320, 389, 411, 437, …


In [27]:
df_vegetation_cover %>%
  group_by(survey_code, species_key) %>%
  # count() %>%
  arrange(survey_code, species_key)

survey_code,plot_code,year,plot_loc,plot_rep,plot_num,detection_type,species_key
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,5
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,37
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,37
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,39
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,39
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,51
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,82
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,84
YVP 10 2017-06-09,YVP 10,2017,NA,NA,10,cover_est,90


In [28]:
# remove duplicate species records per survey
df_vegetation_cover <- df_vegetation_cover %>%
  group_by(survey_code) %>%
  distinct() %>%
  arrange(desc(survey_code), species_key) %>% glimpse()

Rows: 4,933
Columns: 8
Groups: survey_code [175]
$ survey_code    <chr> "YVP NC294 2019-05-09", "YVP NC294 2019-05-09", "YVP N…
$ plot_code      <chr> "YVP NC294", "YVP NC294", "YVP NC294", "YVP NC294", "Y…
$ year           <chr> "2019", "2019", "2019", "2019", "2019", "2019", "2019"…
$ plot_loc       <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N",…
$ plot_rep       <chr> "C", "C", "C", "C", "C", "C", "C", "C", "C", "C", "C",…
$ plot_num       <int> 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294,…
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "c…
$ species_key    <int> 5, 20, 57, 72, 74, 82, 153, 154, 174, 187, 202, 220, 2…


### yvp_additional_species

In [29]:
df_additional_species %>% glimpse()

Rows: 1,280
Columns: 8
$ survey_code    <chr> "YVP 12 2018-07-10", "YVP 12 2018-07-10", "YVP 12 2018…
$ plot_code      <chr> "YVP 12", "YVP 12", "YVP 12", "YVP 12", "YVP 12", "YVP…
$ year           <chr> "2018", "2018", "2018", "2018", "2018", "2018", "2018"…
$ plot_loc       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 12, 12, 12, 12, 12, 246, 246, 246, 246, 571, 571, 571,…
$ detection_type <chr> "supplemental_obs", "supplemental_obs", "supplemental_…
$ species_key    <int> 72, 179, 426, 63, 484, 402, 53, 274, 334, 496, 240, 56…


In [30]:
# remove duplicates per survey
df_additional_species <- df_additional_species %>%
  group_by(survey_code) %>%
  distinct() %>%
  arrange(desc(survey_code), species_key) %>% glimpse()

Rows: 1,275
Columns: 8
Groups: survey_code [178]
$ survey_code    <chr> "YVP NC294 2019-05-09", "YVP NC294 2019-05-09", "YVP N…
$ plot_code      <chr> "YVP NC294", "YVP NC294", "YVP NC294", "YVP NC294", "Y…
$ year           <chr> "2019", "2019", "2019", "2019", "2019", "2019", "2019"…
$ plot_loc       <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N",…
$ plot_rep       <chr> "C", "C", "C", "C", "C", "C", "C", "C", "C", "C", "C",…
$ plot_num       <int> 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294,…
$ detection_type <chr> "supplemental_obs", "supplemental_obs", "supplemental_…
$ species_key    <int> 31, 36, 84, 178, 183, 316, 362, 36, 216, 316, 342, 362…


## Combine dataframes

In [31]:
species_richness <- union(df_vegetation_cover, df_additional_species) %>% glimpse()

Rows: 6,208
Columns: 8
Groups: survey_code [187]
$ survey_code    <chr> "YVP NC294 2019-05-09", "YVP NC294 2019-05-09", "YVP N…
$ plot_code      <chr> "YVP NC294", "YVP NC294", "YVP NC294", "YVP NC294", "Y…
$ year           <chr> "2019", "2019", "2019", "2019", "2019", "2019", "2019"…
$ plot_loc       <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N",…
$ plot_rep       <chr> "C", "C", "C", "C", "C", "C", "C", "C", "C", "C", "C",…
$ plot_num       <int> 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294,…
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "c…
$ species_key    <int> 5, 20, 57, 72, 74, 82, 153, 154, 174, 187, 202, 220, 2…


In [32]:
# look for duplicates
duplicate_detections <- species_richness %>%
  group_by(survey_code) %>%
  count(species_key) %>%
  filter(n > 1) %>%
  arrange(survey_code, species_key) %>% glimpse()

Rows: 104
Columns: 3
Groups: survey_code [62]
$ survey_code <chr> "YVP 10 2018-07-12", "YVP 10 2018-07-12", "YVP 10 2019-07…
$ species_key <int> 16, 433, 16, 220, 433, 308, 220, 51, 51, 439, 576, 211, 2…
$ n           <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, …


## Remove redundant detections

In [33]:
duplicate_detections <- duplicate_detections %>%
  mutate(mask = paste(survey_code, species_key)) %>%
  select(!n) %>%
  ungroup() %>% glimpse()

Rows: 104
Columns: 3
$ survey_code <chr> "YVP 10 2018-07-12", "YVP 10 2018-07-12", "YVP 10 2019-07…
$ species_key <int> 16, 433, 16, 220, 433, 308, 220, 51, 51, 439, 576, 211, 2…
$ mask        <chr> "YVP 10 2018-07-12 16", "YVP 10 2018-07-12 433", "YVP 10 …


In [34]:
# subset surveys with no redundancy
species_richness_x <- species_richness %>%
  ungroup() %>%
  mutate(mask = paste(survey_code, species_key)) %>%
  filter(!mask %in% duplicate_detections$mask) %>%
  arrange(survey_code, species_key) %>% glimpse()

Rows: 6,000
Columns: 9
$ survey_code    <chr> "YVP 10 2017-06-09", "YVP 10 2017-06-09", "YVP 10 2017…
$ plot_code      <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP…
$ year           <chr> "2017", "2017", "2017", "2017", "2017", "2017", "2017"…
$ plot_loc       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10…
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "s…
$ species_key    <int> 5, 37, 39, 51, 72, 82, 84, 90, 153, 163, 169, 187, 202…
$ mask           <chr> "YVP 10 2017-06-09 5", "YVP 10 2017-06-09 37", "YVP 10…


In [35]:
# subset surveys with redundancy
species_richness_y <- species_richness %>%
  ungroup() %>%
  mutate(mask = paste(survey_code, species_key)) %>%
  filter(mask %in% duplicate_detections$mask, detection_type != "supplemental_obs") %>%
  arrange(survey_code, species_key) %>% glimpse()

Rows: 104
Columns: 9
$ survey_code    <chr> "YVP 10 2018-07-12", "YVP 10 2018-07-12", "YVP 10 2019…
$ plot_code      <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP…
$ year           <chr> "2018", "2018", "2019", "2019", "2019", "2017", "2019"…
$ plot_loc       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 10, 10, 10, 10, 10, 113, 12, 144, 144, 144, 144, 181, …
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "c…
$ species_key    <int> 16, 433, 16, 220, 433, 308, 220, 51, 51, 439, 576, 211…
$ mask           <chr> "YVP 10 2018-07-12 16", "YVP 10 2018-07-12 433", "YVP …


In [36]:
# union x and y
species_richness <- union(species_richness_x, species_richness_y)

In [37]:
species_richness <- species_richness %>%
  select(!mask) %>%
  group_by(survey_code) %>%
  arrange(survey_code, species_key) %>% glimpse()

Rows: 6,104
Columns: 8
Groups: survey_code [187]
$ survey_code    <chr> "YVP 10 2017-06-09", "YVP 10 2017-06-09", "YVP 10 2017…
$ plot_code      <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP…
$ year           <chr> "2017", "2017", "2017", "2017", "2017", "2017", "2017"…
$ plot_loc       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_rep       <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", "NA", …
$ plot_num       <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10…
$ detection_type <chr> "cover_est", "cover_est", "cover_est", "cover_est", "s…
$ species_key    <int> 5, 37, 39, 51, 72, 82, 84, 90, 153, 163, 169, 187, 202…


# Join 

## location_position_classification

In [38]:
species_richness <- species_richness %>%
  left_join(df_position_classification, by = c("plot_num" = "grid_point"))

## vegetation_species_metadata

In [39]:
species_richness <- species_richness %>%
  left_join(df_species_metadata, by = c("species_key" = "key_plant_species"))

In [42]:
names(species_richness)

[1] "survey_code"                 "plot_code"                  
 [3] "year"                        "plot_loc"                   
 [5] "plot_rep"                    "plot_num"                   
 [7] "aspect_mean_deg"             "elevation_mean_m"           
 [9] "slope_mean_deg"              "cover_type_2016_gridVeg"    
[11] "type3_vegetation_indicators" "type4_indicators_history"   
[13] "detection_type"              "species_key"                
[15] "key_plant_code"              "plant_name_sci"             
[17] "plant_name_common"           "plant_native_status"        
[19] "plant_life_cycle"            "plant_life_form"

In [43]:
# reorder columns
species_richness <- species_richness %>%
  select(1:6, 9:14, 7, 8, 15:20)

In [44]:
species_richness %>% glimpse()

Rows: 6,104
Columns: 20
Groups: survey_code [187]
$ survey_code                 <chr> "YVP 10 2017-06-09", "YVP 10 2017-06-09",…
$ plot_code                   <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "…
$ year                        <chr> "2017", "2017", "2017", "2017", "2017", "…
$ plot_loc                    <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA",…
$ plot_rep                    <chr> "NA", "NA", "NA", "NA", "NA", "NA", "NA",…
$ plot_num                    <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1…
$ slope_mean_deg              <dbl> 20.794, 20.794, 20.794, 20.794, 20.794, 2…
$ cover_type_2016_gridVeg     <chr> "non-irrigated grasslands", "non-irrigate…
$ type3_vegetation_indicators <chr> "wooded draw", "wooded draw", "wooded dra…
$ type4_indicators_history    <chr> "wooded draw", "wooded draw", "wooded dra…
$ detection_type              <chr> "cover_est", "cover_est", "cover_est", "c…
$ species_key                 <int> 5, 37, 39, 51, 72, 82, 84, 90, 153, 163, …
$ 

# Output

In [45]:
# output update 2020-07-21
write_csv(species_richness, path = "yvp_species_richness-WRANGLE.csv")